# <font color="f0f000">R O B O C L A W</font>
$\rightarrow$ advanced practical use<hr>
Written by: Quentin Demory<br>
August 2020 @ SSRS<br>
MIT license<hr>

In [1]:
from roboclaw_3 import Roboclaw

In [2]:
# Open serial port - Rpi
rc = Roboclaw("/dev/ttyACM0", 115200)
rc.Open()

1

In [3]:
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 200000, 10000, 1)

True

In [4]:
rc.ReadEncM1(129)

(1, 9997, 128)

In [5]:
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 200000, 0, 1)

True

In [6]:
rc.ReadEncM1(129)

(1, 4, 130)

In [11]:
rc.ReadMainBatteryVoltage(129)

(1, 237)

In [41]:
print("Main battery @ {} V".format(rc.ReadMainBatteryVoltage(129)[1]/10))

Main battery @ 23.7 V


In [13]:
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 200000, 10000, 1)
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 200000, 0, 0)

True

In [14]:
rc.ReadM1VelocityPID(129)

[1, 0.7190093994140625, 0.041595458984375, 0.0, 59437]

In [15]:
rc.ReadM1PositionPID(129)

[1, 78.0634765625, 0.0, 383.791015625, 0, 4, 0, 10000]

In [16]:
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 200000, 10000, 1)
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 200000, 0, 0)
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 200000, 5000, 0)
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 200000, 7500, 0)
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 200000, 10000, 0)
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 200000, 0, 0)
rc.ReadBuffers(129)

(1, 5, 128)

In [17]:
rc.ReadBuffers(129)

(1, 128, 128)

In [36]:
rc.ForwardM1(129, 63)

True

In [38]:
rc.ForwardM1(129, 0)

True

In [39]:
rc.ReadEncM1(129)

(1, 200527, 128)

In [40]:
import time
start_time = time.time()
rc.SpeedAccelDeccelPositionM1(129, 400000, 10000, 200000, 0, 1)
rc.SpeedAccelDeccelPositionM1(129, 0, 0, 0, 0, 0)
time.sleep(0.2)

while rc.ReadSpeedM1(128)[1] != 0:
        time.sleep(0.02)

print("travel back to 0 from a random position: {} s".format(time.time()-start_time))

travel back to 0 from a random position: 3.230962038040161 s


In [42]:
rc.ReadCurrents(129)

(1, 2, 23)

In [43]:
current_read_channel1 = rc.ReadCurrents(129)[1]/10
current_read_channel2 = rc.ReadCurrents(129)[2]/10

print(current_read_channel1)
print(current_read_channel2)

0.2
1.5


In [45]:
temp_channel1 = rc.ReadTemp(129)[1]/10
temp_channel2 = rc.ReadTemp2(129)[1]/10

print(temp_channel1)
print(temp_channel2)

29.1
29.1


In [46]:
rc.ReadError(129)

(1, 0)

In [47]:
rc.comport

'/dev/ttyACM0'

In [48]:
rc.rate

115200

In [55]:
rc.SpeedM1(129, 20000)

True

In [56]:
rc.SpeedM1(129, 0)

True

In [57]:
rc.ReadEncM1(129)

(1, 553902, 131)

In [58]:
rc.SpeedDistanceM1(129, 50000, 555000, 1)

True

In [60]:
rc.ReadEncM1(129)

(1, 1133971, 128)

In [61]:
rc.SetEncM1(129, 0)
rc.ReadEncM1(129)

(1, 0, 128)

In [62]:
rc.SpeedAccelDeccelPositionM1(129, 500000, 40000, 250000, 12000, 1)

True

In [63]:
rc.ReadEncM1(129)

(1, 9996, 129)

In [64]:
rc.ReadMainBatteryVoltage(129)[1]/10

23.7

In [65]:
rc.SpeedAccelDeccelPositionM1(129, 500000, 40000, 250000, -10000, 1)

True

In [66]:
rc.ReadEncM1(129)

(1, 4, 130)

In [67]:
rc.SpeedAccelDistanceM1(129, 400000, -40000, 10000, 1)

True

In [68]:
rc.ReadEncM1(129)

(1, -30013, 130)